In [ ]:
import numpy as np
import sys
sys.path.append('/home/marcobonici/Desktop/CosmologicalEmulators/PyEffort/src') # write the path were you put PyCapse
import PyEffort
import matplotlib.pyplot as plt

In [ ]:
emu_xi = PyEffort.load_emu("/home/marcobonici/Desktop/CosmologicalEmulators/trained_emulators/test_train_velocileptor_xi/trained_velocileptor_xi_l_0_z1.bson")
input_cosmo = np.array([3.037842, 0.9553725, 0.67123, 0.0230248, 0.142591])
Xil_test = PyEffort.compute_Xil(input_cosmo, np.ones(6), emu_xi)

In [ ]:
plt.plot(emu_xi.rgrid, Xil_test)

In [ ]:
emu_P = PyEffort.load_emu("/home/marcobonici/Desktop/CosmologicalEmulators/trained_emulators/test_train_velocileptor/trained_velocileptor_l_0_z1.bson")
input_cosmo = np.array([3.037842, 0.9553725, 0.67123, 0.0230248, 0.142591])
Pl_test = PyEffort.compute_Pl(input_cosmo, np.ones(11), emu_P)

In [ ]:
plt.plot(emu_P.kgrid, emu_P.kgrid*Pl_test)

In [ ]:
%timeit PyEffort.compute_Xil(input_cosmo, np.ones(6), emu_xi)

In [ ]:
%timeit PyEffort.compute_Pl(input_cosmo, np.ones(11), emu_P)

In [ ]:
emu_P_pybird = PyEffort.load_emu("/home/marcobonici/Desktop/CosmologicalEmulators/trained_emulators/trained_effort_emulators/blinded_challenge/PyBird_061_10000_optiresum_final/emulator_PyBird_w0wanucdm_monopole.bson")

In [ ]:
Pl_test = PyEffort.compute_Pl(np.array([3.0, 67., 0.32]), np.ones(7), 1.5, emu_P_pybird)

In [ ]:
f = PyEffort.compute_fz(0.61, 0.32, -1., 0.)

In [ ]:
%timeit PyEffort.compute_Pl(np.array([3.0, 67., 0.32]), np.ones(7), f, emu_P_pybird)

In [ ]:
plt.plot(emu_P_pybird.P11.kgrid, emu_P_pybird.P11.kgrid*Pl_test)